## Stress-strain rate and pseudo-plasticity

The velocity gradient tensor is:

$$ \nabla u_{ij} = \frac{\partial uj}{\partial u_i}$$

The velocity gradient tensor can be decomposed int a symmetric and antisymmetric part:

$$\nabla u_{ij} = \frac{1}{2}[\nabla u + \nabla u^T] + \frac{1}{2}[\nabla u - \nabla u^T]$$

The symmetric part, we call the strain the tensor:

$$\epsilon_{ij} = \frac{1}{2}[\nabla u + \nabla u^T]$$

Newtonian viscosity is a material model where there exists a __linear__ relationship between the stress tensor (symmetric by definition), and the  symmetric part of the velocity gradient tensor, i.e. the strain rate tensor:

$$\tau_{ij} = 2\eta\epsilon_{ij} = \eta[\nabla u + \nabla u^T]$$

The factor of two is arbitary, and in some definitions is not used. See http://www.earth.lsa.umich.edu/~keken/thesis/chap2.pdf for a description of alternative forms.

In 2 dimensions, the strain rate tensor is 

$$\epsilon_{ij} = \frac{1}{2}[\nabla u + \nabla u^T] =\left[
  \begin{array}{ c c }
     \frac{\partial{v_x}}{\partial_x} & \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) \\
     \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) & \frac{\partial{v_y}}{\partial_y} 
  \end{array} \right]
$$

While the anti-symmetric vorticity tensor is:

$$ \omega_{ij} = \frac{1}{2}[\nabla u - \nabla u^T] =\left[
  \begin{array}{ c c }
     0 & \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} - \frac{\partial{v_y}}{\partial{x}}\right) \\
     \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} - \frac{\partial{v_y}}{\partial{x}}\right) & 0
  \end{array} \right]
$$


The Drucker-Prager yield criterion can be expressed as:

$$\tau_{II} \leq C + p\mu = \sigma_y$$

where, $\tau_{II}$ is the second invariant of the deviatoric stress tensor.

The second invariant of the deviatoric stress tensor is derived from the second invariant of the strain rate tensor:


$$\tau_{II}  = 2\mu \dot \epsilon_{II}$$

Where:

$$\epsilon_{II} = \sqrt{\frac{1}{2}\epsilon_{ij}\epsilon_{ij} }$$

The factor of $1/2$ is not always used, but it is in in Underworld; i.e. this is the form of the second invariant used by functions like: 

```
fn.tensor.second_invariant(uw.function.tensor.symmetric(velocityField.fn_gradient))


```
The term $\epsilon_{ij}\epsilon_{ij} $ represents a tensor inner product (so it returns a scalar). In 2d, this is:







$$\epsilon_{ij}\epsilon_{ij}  = $$

$$\left[
  \begin{array}{ c c }
     \frac{\partial{v_x}}{\partial_x} & \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) \\
     \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) & \frac{\partial{v_y}}{\partial_y} 
  \end{array} \right]
  \left[
  \begin{array}{ c c }
     \frac{\partial{v_x}}{\partial_x} & \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) \\
     \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right) & \frac{\partial{v_y}}{\partial_y} 
  \end{array} \right]
$$

$$ = \frac{\partial{v_x}}{\partial_x}^2  +  \frac{1}{2}\left(\frac{\partial{v_x}}{\partial{y}} + \frac{\partial{v_y}}{\partial{x}}\right)^2 + \frac{\partial{v_y}}{\partial_y}^2 $$

Some models will use a slightly different form of the the second invariant of the strain rate tensor i.e:

$$\epsilon_{II} = \sqrt{\epsilon_{ij}\epsilon_{ij} }$$

If this how a material model has been implemented, the Underworld2 second invariant function must be modified:

$$\epsilon_{IIuw} = \sqrt{\frac{1}{2}} \times\epsilon_{II} $$

where:

$$\epsilon_{II} = \sqrt{\epsilon_{ij}\epsilon_{ij} }$$



In the psuedo-plastic approach, whenever the second invariant of the viscous stress tensor is above the yield limit (criterion), the viscosity is modified so that the stress remain at or beneath the limit. This effective visocosity is:

$$\eta_{eff} = \frac{\sigma_y}{2\times \epsilon_{II} } $$

The factor of two here is important insofar as keeping our definitions consistent. 

## Underworld2

Underworld2 enables psuedo-plastic rheology through the _stress_limiting_viscosity_ function;

`fn.rheology.stress_limiting_viscosity(fn_stress, fn_stresslimit, fn_inputviscosity)`

In this example: 

* fn_stress: full stress tensor (expressed in terms of the strain rate tensor)
* fn_stresslimit: stress limiting value, or function
* fn_inputviscosity: non-plastic rheology

The function returns an effective viscosity $\eta_{eff}$ wherever the stress limit is reached. $\eta_{eff}$ is set to:

`fn_inputviscosity*_fn_stresslimit/secondInvFn`

where:

`secondInvFn = _tensor.second_invariant(self._fn_stress)`

or

$$\frac{\eta \times \sigma_y}{\tau_{II}}$$

Which is exactly the same as:

$$\frac{\sigma_y}{2\times \epsilon_{II} } $$

In operation, $\tau_{II}$ is only defined in reference to $\epsilon_{II}$, so the forms are exactly equivalent.

